# 影像分割 Segmentation

分割（Segmentation）是指在影像處理中，將影像劃分為多個不同的區域或區塊的過程。這些區域可以是具有相似特徵的像素集合，例如顏色、亮度、紋理等。分割技術的目標是將影像中的每個像素分配到相應的區域，從而使得影像中的不同物體或結構可以被區分開來。

在影像分割中，常見的方法包括：

1. **基於閾值的分割**：根據像素的灰度值或顏色值，將影像劃分為前景和背景兩個區域。
  
2. **區域生長**：從種子點開始，根據像素的相似性，將相鄰的像素逐漸合併成為同一區域。
  
3. **邊緣檢測**：檢測影像中的邊緣，從而將影像劃分為不同的物體或結構。

分割技術在計算機視覺和影像處理中具有廣泛的應用，包括但不限於：

1. **醫學影像分析**：如MRI、CT影像中的器官和組織分割，用於疾病診斷和治療規劃。

2. **地理信息系統（GIS）**：將衛星影像或地圖影像中的不同地物區分開來，用於土地利用分析、城市規劃等。

3. **自動駕駛**：將道路影像中的行車線、車輛和行人等分割開來，用於自駕車的路徑規劃和障礙物檢測。

總之，分割是影像處理中將影像劃分為多個區域的過程，其應用涉及多個領域，並在計算機視覺和圖像分析中扮演著重要角色。

In [ ]:
!gdown '1y_zwNgVOxwxSus-CvQRzKsR5PJ-1p9nz' --output lesson9.zip
!unzip lesson9.zip



- 如果您想在自己的電腦上執行此程式碼，您可以安裝以下內容:

```
    !pip install transformers
    !pip install gradio
    !pip install timm
    !pip install torchvision
```

In [ ]:
!pip install -q -U gradio timm

- 這是一些抑制警告訊息的程式碼。

這段程式碼是從`transformers`庫中引入`logging`模組，然後設置日誌輸出的細節程度為錯誤級別（error）。這樣做的目的通常是為了避免顯示過多的日誌訊息，只保留錯誤和更高級別的訊息，以便更容易追蹤和處理錯誤。

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

### 使用 SAM 產生遮罩 mask-generation

[Segment Anything Model (SAM)](https://segment-anything.com) 模型由 Meta AI 發布。

關於資訊 [Zigeng/SlimSAM-uniform-77](https://huggingface.co/Zigeng/SlimSAM-uniform-77)

In [ ]:
from tramsformers import pipeline
sam_pips = pipeline('mask-generation', 'Zigeng/SlimSAM-uniform-77')

from PIL import Image
raw_iamge = Image.open('/content/meta_liamas.jpg')
raw_image.resize(720, 375)

output = sam_pipe(raw_iamge, points_per_batch = 16)

from helper import show_pipe_masks_on_image
show_pipe_masks_on_iamge(raw_iamge, output)

len(output['masks'])
output['scores']

- 運行這將需要一些時間
-  `points_per_batch` 的值越高，管道推理就越高效
_注意：_ 運行此程式碼時獲得的分段顏色可能與您在影片中看到的顏色不同。

### 更快的推理：推理影像和單點

In [ ]:
from transformers import SamModel, SamProcessor
model = SmaModel.from_pretrained('Zigeng/SlimSAM-uniform-77')
processor =  SmaProcessor.from_pretrained('Zigeng/SlimSAM-uniform-77')
model
                                          
raw_iamge.resize((720, 375))
input_points = [[[400, 200]]]
inputs = processor(raw_image, input_points = input_points, return_tensors = 'pt')

import torch
with torch.no_grad():  outputs = model(''inputs)
outputs

predicted_masks = processor.image_processor.post_process_masks(
    outputs.pred_masks,
    inputs["original_sizes"],
    inputs["reshaped_input_sizes"]
)
len(predicted_masks)

predicted_mask = predicted_masks[0]
predicted_mask.shape
outputs.iou_scores

from helper import show_mask_on_image
for i in range(3):
    show_mask_on_image(raw_image, predicted_mask[:, i])
    

- 分割安德魯穿的藍色襯衫。
- 給出該區域中的任何單一 2D 點（藍色襯衫）。
- 使用影像和單點建立輸入。
- `return_tensors="pt"` 表示傳回 PyTorch 張量。
- 給定輸入，從模型中取得輸出。
`predicted_masks`的長度對應於輸入中使用的影像數量。
- 檢查第一個 ([0]) 預測遮罩的大小

## 使用 DPT 進行深度估計 depth-estimation

- Ranftl 等人在論文 [Vision Transformers for Dense Prediction](https://arxiv.org/abs/2103.13413) 中介紹了此模型。 (2021) 並首次在 [isl-org/DPT](https://github.com/isl-org/DPT) 中發布。

In [ ]:
depth_estimator = pipeline('depth-estimation','Intel/dpt-hybrid-midas')

raw_image = Image.open('/content/gradio_tamagochi_vienna.png')
raw_image.resize((806, 621))

output = depth_estimator(raw_image)
output

output["predicted_depth"].shape

output["predicted_depth"].unsqueeze(1).shape

# resize the prediction 調整預測大小
prediction = torch.nn.functional.interpolate(
    output["predicted_depth"].unsqueeze(1),
    size=raw_image.size[::-1],
    mode="bicubic",
    align_corners=False,
)
prediction.shape

raw_image.size[::-1]

prediction

import numpy as np
# normalize the prediction 標準化預測
output = prediction.squeeze().numpy() #降1個維度且轉成numpy數組
formatted = (output * 255 / np.max(output)).astype("uint8") #對深度值進行歸一化處理
depth = Image.fromarray(formatted) #深度轉換為PIL圖像
depth

關於資訊 ['Intel/dpt-hybrid-midas'](https://huggingface.co/Intel/dpt-hybrid-midas)
- - 如果您想產生此圖片或類似的圖像，請查看 [Gradio](https://www.deeplearning.ai/short-courses/building-generative-ai-applications-with-gradio/) 上的短期課程 並轉到“圖像生成應用程式 Image Generation App”課程。
- 對輸出影像進行後處理，將其大小調整為原始影像的大小。
- 將預測張量（0 到 255 之間）進行歸一化，以便可以顯示它們。

這段程式碼對模型預測的深度圖進行了後續處理和轉換成可視化的影像。以下是對程式碼的註解：

```
import numpy as np
from PIL import Image

# prediction是經過插值後的深度圖tensor，使用squeeze函數將其壓縮維度，將1維的channel維度去除，得到二維的深度圖tensor
# 使用numpy()方法將tensor轉換為NumPy陣列
output = prediction.squeeze().numpy()

# 對深度值進行歸一化處理，將深度值映射到0到255之間
# 將深度值乘以255除以最大深度值，然後轉換為整數型別（uint8）
# 這樣處理後深度圖的深度值範圍就變成了0到255之間，便於可視化顯示
formatted = (output * 255 / np.max(output)).astype("uint8")

# 將處理過的深度圖轉換為PIL圖像對象，以便後續的顯示或保存
depth = Image.fromarray(formatted)

```

總的來說，這段程式碼將模型預測的深度圖進行了後續的處理，使其變得更容易理解和顯示。

## 使用 Gradio 進行演示

### 故障排除提示
- 請注意，在課堂上，您可能會看到用於建立 Gradio 應用程式的程式碼無限期地運行。
   - 這是特定於該課堂環境的，當它同時為許多學習者提供服務時，如果您在自己的電腦上執行此程式碼，您將不會遇到此問題。
- 若要解決此問題，請重新啟動核心（選單「核心」->「重新啟動核心」）並從課程開始重新執行實驗室中的程式碼。

### 更快的推理：推理影像和單點

In [ ]:
import os
import gradio as gr
from transformers import pipeline

def launch(input_image):
  out = depth_estimator(input_image)
  # resize the prediction調整預測大小
  prediction = torch.nn.functional.interpolate(
    out["predicted_depth"].unsqueeze(1),
    size=input_image.size[::-1],
    mode="bicubic",
    align_corners=False,)

  # normalize the prediction 標準化預測
  output = prediction.squeeze().numpy() #降1個維度且轉成numpy數組
  formatted = (output * 255 / np.max(output)).astype("uint8") #對深度值進行歸一化處理
  depth = Image.fromarray(formatted) #深度轉換為PIL圖像
  return depth

iface = gr.Interface(launch, gr.Image(type="pil"), gr.Image(type="pil"))
iface.launch(share=True)

In [ ]:
iface.close()